In [ ]:
# !git clone -q https://github.com/tilt/RasterFairy.git
# !cd RasterFairy && pip install -q .

In [2]:
import os
import shutil

# 원본 데이터 경로와 재구성된 데이터 저장 경로
original_dataset_path = "/home/work/ModuInterior/dataset/bonn-custom/houzz"
restructured_dataset_path = "/home/work/ModuInterior/dataset/restructured"

# 클래스별로 폴더 생성 및 이미지 이동
os.makedirs(restructured_dataset_path, exist_ok=True)
for class_folder in os.listdir(original_dataset_path):  # beds, chairs 등
    class_path = os.path.join(original_dataset_path, class_folder)
    if os.path.isdir(class_path):
        target_class_path = os.path.join(restructured_dataset_path, class_folder)
        os.makedirs(target_class_path, exist_ok=True)
        for style_folder in os.listdir(class_path):  # Contemporary, Modern 등
            style_path = os.path.join(class_path, style_folder)
            if os.path.isdir(style_path):
                for image_file in os.listdir(style_path):  # 이미지 파일
                    if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                        src_path = os.path.join(style_path, image_file)
                        dst_path = os.path.join(target_class_path, image_file)
                        shutil.copy(src_path, dst_path)

print(f"데이터가 {restructured_dataset_path}에 재구성되었습니다.")


데이터가 /home/work/ModuInterior/dataset/restructured에 재구성되었습니다.


In [8]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

# 재구성된 데이터 경로
dataset_path = "/home/work/ModuInterior/dataset/restructured"

# 이미지 전처리 및 데이터 로드
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset = ImageFolder(root=dataset_path, transform=transform)
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

# 클래스 및 데이터 확인
print(f"클래스 수: {len(dataset.classes)}")
print(f"클래스 목록: {dataset.classes}")


FileNotFoundError: Found no valid file for the classes hvt_db. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:
import torch
import torchvision.models as models

# 사전 학습된 모델로 특징 추출
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # 마지막 분류 헤드 제거
model.eval().cuda()

# 데이터셋의 임베딩 추출
embeddings = []
labels = []
with torch.no_grad():
    for images, targets in data_loader:
        images = images.cuda()
        features = model(images).squeeze(-1).squeeze(-1)
        embeddings.append(features.cpu())
        labels.append(targets)

# 임베딩 및 라벨 결합
embeddings = torch.cat(embeddings, dim=0)
labels = torch.cat(labels, dim=0)

In [ ]:
import umap
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Circle

# UMAP으로 하이퍼볼릭 공간 매핑
mapper = umap.UMAP(output_metric="hyperboloid", random_state=1337)
path2d = mapper.fit_transform(embeddings.numpy())
x, y = path2d[:, 0], path2d[:, 1]
z = (1 + x ** 2 + y ** 2) ** 0.5
disk_x = x / (1 + z)
disk_y = y / (1 + z)

# 시각화
fig, ax = plt.subplots(figsize=(10, 10))
scatter = ax.scatter(disk_x, disk_y, c=labels.numpy(), cmap="tab10", s=7, alpha=0.75)
boundary = Circle((0, 0), 1, fc="none", ec="k")
ax.add_patch(boundary)
ax.axis("off")
plt.colorbar(scatter, label="Class Labels")
plt.show()


In [ ]:
import rasterfairy
from PIL import Image

# RasterFairy로 2D 배치 생성
xy = np.stack([disk_x, disk_y], axis=1)
grid_xy = rasterfairy.transformPointCloud2D(xy)
grid_w, grid_h = grid_xy[1]
img_size = 50  # 이미지 크기 설정

# 배경 캔버스 생성
background = Image.new("RGB", (grid_w * img_size, grid_h * img_size), (255, 255, 255))

# 이미지 배치
for i in range(len(grid_xy[0])):
    x, y = grid_xy[0][i]
    img, _ = dataset[i]
    img = transforms.ToPILImage()(img)
    img = img.resize((img_size, img_size))
    background.paste(img, (int(x * img_size), int(y * img_size)))

# 결과 저장
background.save("/home/work/ModuInterior/results/umap_visualization.jpg", "JPEG", quality=90, optimize=True)
